# Generate SPSS files, for 'normal' (IR) NVivo files


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import re
from string import printable
import ftfy
from scipy.stats import chi2_contingency


In [2]:
# widen display to allow long text strings to be more visible on screen
pd.set_option('display.max_colwidth',2000)

In [3]:
%pwd

'\\\\ads\\filestore\\Deaf_Studies\\Deaf\\nfrc\\LeDeR confidential\\Data Entry\\Avon\\reviews_leder_revised_codes\\python_notebooks'

# read in master nvivo_df

In [4]:
master_nvivo_df = pd.read_pickle('master_nvivo_df.pickle')

# Create sub-sample
based on document type

In [5]:
normal_df = master_nvivo_df[master_nvivo_df.case_file_type_atext=='Normal']

In [6]:
list(normal_df.columns)

['AAA_screening',
 'AAA_screening_atext',
 'AAA_screening_no',
 'AAA_screening_no_atext',
 'AAA_screening_yes',
 'AAA_screening_yes_atext',
 'AP_agencies',
 'AP_agencies_atext',
 'AP_completed_DNACPR',
 'AP_completed_DNACPR_atext',
 'AP_completed_EOL_care',
 'AP_completed_EOL_care_atext',
 'AP_completed_LD_awareness',
 'AP_completed_LD_awareness_atext',
 'AP_completed_MCA',
 'AP_completed_MCA_atext',
 'AP_completed_agencies',
 'AP_completed_agencies_atext',
 'AP_completed_care_coordination',
 'AP_completed_care_coordination_atext',
 'AP_completed_documentation',
 'AP_completed_documentation_atext',
 'AP_completed_family',
 'AP_completed_family_atext',
 'AP_completed_other',
 'AP_completed_other_atext',
 'AP_completed_reasonable_adjustments',
 'AP_completed_reasonable_adjustments_atext',
 'AP_completed_role_paid_carers',
 'AP_completed_role_paid_carers_atext',
 'AP_person',
 'AP_person_atext',
 'AP_raised_DNACPR',
 'AP_raised_DNACPR_atext',
 'AP_raised_EOL_care',
 'AP_raised_EOL_care_at

# List of Downs people for Rachel

In [7]:
downs_ids = normal_df[normal_df.Clinical_downs==1].N_ID
downs_ids.to_csv('downs_ids.csv',index=False)

In [8]:
mels_list = pd.read_csv('mels_nids.csv',dtype={'nid':str})
print(mels_list.tail())

         nid
31  25287401
32  25290145
33  25296025
34  25297187
35  23293351


In [9]:
pd.merge(mels_list,normal_df,left_on='nid',right_on='N_ID',how='left')[['N_ID','coder_atext']]

,N_ID,coder_atext
0,20298380,nick
1,20972552,nick
2,23041548,ann
3,25130552,karen
4,25150712,karen
5,25151846,nick
6,25163151,karen
7,NaN,NaN
8,25191732,kamila
9,25198851,karen


In [10]:
normal_df_sorted = normal_df.sort_values(['N_ID','number_codes'])
# get last in any list of same IDs to select the review with most codes if any were coded by 2 or more team
normal_df_no_dups = normal_df_sorted.groupby('N_ID').last().reset_index()



In [11]:
len(normal_df_no_dups)

1372

In [12]:
# save this final df to text file for SPSS use
normal_df_no_dups.to_csv('normal_nvivo_df_spss.txt',index=False,sep='\t')

In [13]:
index_of_big_cell = normal_df_no_dups[normal_df_no_dups['Medical_histories_atext'].str.len()>32750].index.values[0]
big_cell_text = normal_df_no_dups[normal_df_no_dups['Medical_histories_atext'].str.len()>32750].Medical_histories_atext.values[0]


In [14]:
index_of_big_cell
normal_df_no_dups.loc[index_of_big_cell,'Medical_histories_atext'] = big_cell_text[:32760]

In [15]:
# generate spss syntax file to load nvivo data into spss
# what are the variables?
# if variable ends with '_atext' then get length and set to string

# edited 30/5/19 to ensure that SPSS maximum length of string variable (32767 chars) is not exceeded

required_columns =  list(normal_df_no_dups.columns)
spps_syntax_for_normal_nvivo_vars = ''
for var in required_columns:
    if var.endswith('_atext'):
        print(var)
        max_text_length = str(int(normal_df_no_dups[var].str.len().max()))
        print('max = ',max_text_length)
        spps_syntax_for_normal_nvivo_vars = spps_syntax_for_normal_nvivo_vars + var + '  A' + max_text_length + '  \n'
    else:
        spps_syntax_for_normal_nvivo_vars = spps_syntax_for_normal_nvivo_vars + var +' F3.1  \n'
        
print(spps_syntax_for_normal_nvivo_vars)

AAA_screening_atext
max =  69
AAA_screening_no_atext
max =  197
AAA_screening_yes_atext
max =  52
AP_agencies_atext
max =  4462
AP_completed_DNACPR_atext
max =  387
AP_completed_EOL_care_atext
max =  655
AP_completed_LD_awareness_atext
max =  1268
AP_completed_MCA_atext
max =  1069
AP_completed_agencies_atext
max =  2289
AP_completed_care_coordination_atext
max =  1123
AP_completed_documentation_atext
max =  1071
AP_completed_family_atext
max =  0
AP_completed_other_atext
max =  1693
AP_completed_reasonable_adjustments_atext
max =  1221
AP_completed_role_paid_carers_atext
max =  1076
AP_person_atext
max =  1136
AP_raised_DNACPR_atext
max =  1374
AP_raised_EOL_care_atext
max =  2379
AP_raised_LD_awareness_atext
max =  4064
AP_raised_MCA_atext
max =  4327
AP_raised_agencies_atext
max =  8946
AP_raised_care_coordination_atext
max =  3718
AP_raised_documentation_atext
max =  2003
AP_raised_family_atext
max =  1821
AP_raised_other_atext
max =  3519
AP_raised_reasonable_adjustments_atext
max

max =  893
LD_severity_mild_atext
max =  382
LD_severity_moderate_atext
max =  342
LD_severity_profound_multiple_atext
max =  301
LD_severity_severe_atext
max =  504
Last_dental_check_inside_1_year_atext
max =  456
Last_dental_check_over_1_year_atext
max =  164
Last_health_check_date_atext
max =  654
Last_health_check_never_atext
max =  50
Last_health_check_over_1_year_atext
max =  738
Last_health_check_within_1_month_atext
max =  217
Last_health_check_within_1_year_atext
max =  6829
Last_health_check_within_3_months_atext
max =  423
Last_health_check_within_6_months_atext
max =  120
Last_meds_review_6_months_atext
max =  897
Last_meds_review_atext
max =  734
Last_meds_review_month_atext
max =  716
Last_meds_review_over_year_atext
max =  654
Last_meds_review_quarter_atext
max =  657
Last_meds_review_week_atext
max =  665
Last_meds_review_year_atext
max =  386
Last_risk_assessment_over_1_year_atext
max =  428
Last_risk_assessment_within_1_month_atext
max =  737
Last_risk_assessment_with

max =  679
Reasonable_adjustments_desired_home_visit_atext
max =  520
Reasonable_adjustments_desired_other_atext
max =  2613
Reasonable_adjustments_flexibility_on_appts_atext
max =  1788
Reasonable_adjustments_helpers_in_hospital_atext
max =  663
Reasonable_adjustments_home_visit_atext
max =  955
Reasonable_adjustments_made_a_quiet_space_atext
max =  936
Reasonable_adjustments_made_atext
max =  1473
Reasonable_adjustments_made_communication_aids_atext
max =  1182
Reasonable_adjustments_made_flexibility_on_appts_atext
max =  844
Reasonable_adjustments_made_helpers_in_hospital_atext
max =  1322
Reasonable_adjustments_made_home_visit_atext
max =  1281
Reasonable_adjustments_made_other_atext
max =  2397
Reasonable_adjustments_quiet_space_atext
max =  903
Regurgitation_reflux_problems_atext
max =  617
Residential_home_LD_atext
max =  1000
Residential_home_atext
max =  942
Residential_home_dementia_atext
max =  349
Residential_home_generic_atext
max =  501
Residential_home_some_LD_atext
max 

In [16]:
# writing SPSS syntax file to load data.
# note that there must be no whitespaces after the final backslash for each line, as this will cause an error to be thrown

file=open('nvivo_normal_spss_sytax.sps','w')
file.write('* Encoding: UTF-8.\n')
file.write('PRESERVE. \nSET DECIMAL DOT.\n')
file.write('GET DATA  /TYPE=TXT \n \
  /FILE="\\\\ads\\filestore\\Deaf_Studies\\Deaf\\nfrc\LeDeR confidential\\Data Entry\\Avon\\reviews_leder_revised_codes\\python_notebooks\\normal_nvivo_df_spss.txt" \n \
  /ENCODING=\'UTF8\' \n \
  /DELCASE=LINE \n \
  /DELIMITERS="\\t" \n \
  /ARRANGEMENT=DELIMITED \n \
  /FIRSTCASE=2 \n \
  /DATATYPEMIN PERCENTAGE=95.0 \n \
  /VARIABLES= \n ')

file.write(spps_syntax_for_normal_nvivo_vars)



file.write('/MAP. \n \
RESTORE. \n \
CACHE. \n \
EXECUTE. \n \
DATASET NAME nvivo_data_normal_reviews WINDOW=FRONT. \n')

file.close()

In [17]:
%pwd

'\\\\ads\\filestore\\Deaf_Studies\\Deaf\\nfrc\\LeDeR confidential\\Data Entry\\Avon\\reviews_leder_revised_codes\\python_notebooks'

# Obtain possibly significant correlations:
between target variable and all others (where p-vale <0.05)

In [18]:
# function to check if at least five entries in each cell of the crosstab
def check_if_valid_crosstab(a_crosstab):
    z = a_crosstab > 5
    z1 = z.values.tolist() # make df a list of lists
    flat_list = [item for sublist in z1 for item in sublist]
    validity = all(flat_list)
    return(validity)
    

In [19]:
numeric_columns_only_df = normal_df_no_dups.select_dtypes('float64')
for n in range(len(numeric_columns_only_df.columns)):
    x = pd.crosstab(normal_df_no_dups.Clinical_pneumonia_aspiration, numeric_columns_only_df.iloc[:,n])
    y=check_if_valid_crosstab(x)
    if y:
        z = chi2_contingency(x)
        p_value = z[1]
        if p_value < 0.05:
            print(x.index.name,' vs ',x.columns.name, ': ',round(p_value,3))


Clinical_pneumonia_aspiration  vs  AP_raised_EOL_care :  0.025
Clinical_pneumonia_aspiration  vs  AP_raised_care_coordination :  0.037
Clinical_pneumonia_aspiration  vs  Acm_TOD_dur_LT_5years :  0.04
Clinical_pneumonia_aspiration  vs  Acm_TOD_dur_cat :  0.002
Clinical_pneumonia_aspiration  vs  Agency_safeguarding_referral :  0.014
Clinical_pneumonia_aspiration  vs  Any_aids_employed :  0.0
Clinical_pneumonia_aspiration  vs  Aspiration_before_last_year :  0.037
Clinical_pneumonia_aspiration  vs  Aspiration_within_last_year :  0.0
Clinical_pneumonia_aspiration  vs  Blood_pressure_high :  0.025
Clinical_pneumonia_aspiration  vs  Blood_pressure_low :  0.0
Clinical_pneumonia_aspiration  vs  Cancer :  0.0
Clinical_pneumonia_aspiration  vs  Cancer_main_symptom :  0.0
Clinical_pneumonia_aspiration  vs  Care_plan_EOL :  0.0
Clinical_pneumonia_aspiration  vs  Care_plan_dementia :  0.019
Clinical_pneumonia_aspiration  vs  Care_plan_epilepsy :  0.027
Clinical_pneumonia_aspiration  vs  Choking_with

In [20]:
normal_df_no_dups.coder_atext.unique()

array(['nick', 'karen', 'ann', 'nicholasm', 'kamila'], dtype=object)

In [21]:
x=pd.DataFrame(numeric_columns_only_df.sum(axis=0))
x.to_csv('numbers_of_each_code.csv')

In [22]:
x

,0
AAA_screening,1.0
AAA_screening_no,1.0
AAA_screening_yes,2.0
AP_agencies,39.0
AP_completed_DNACPR,4.0
AP_completed_EOL_care,6.0
AP_completed_LD_awareness,19.0
AP_completed_MCA,20.0
AP_completed_agencies,38.0
AP_completed_care_coordination,30.0


## print datetime to confirm run completion time, and then exit

In [23]:
import datetime
print('JOB COMPLETE AT: ',str(datetime.datetime.now()).split('.')[0])

JOB COMPLETE AT:  2019-07-04 10:54:57
